# Imports e definições de funções

In [ ]:
# !pip install tensorflow
# !pip install keras
!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import optuna
import json
import sklearn
import numpy as np
import pandas as pd
from numpy import loadtxt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow import keras
from keras import optimizers
from keras import layers
from keras.layers import Dropout, Conv1D, MaxPooling1D, Flatten, Dense, SpatialDropout1D, BatchNormalization
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam, Adadelta, SGD, RMSprop, Adagrad, Adamax, Nadam, Ftrl
from keras.models import Sequential
from keras.layers import Embedding

from mlxtend.plotting import plot_confusion_matrix
from keras.callbacks import Callback, ModelCheckpoint, CSVLogger
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
from keras.callbacks import EarlyStopping

In [ ]:
def tokenize(tweets_list):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(tweets_list)
    return tokenizer.texts_to_sequences(tweets_list), tokenizer

In [ ]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            self.score = roc_auc_score(self.y_val, y_pred)
            self.false_positive_rate, self.true_positive_rate, _ = roc_curve(self.y_val, y_pred)

# Carrega arquivo de saída (resultados)

In [ ]:
REDE = 'LSTM'

BASE_NAME = 'BASE 1'
#BASE_NAME = 'BASE 2'

# Carrega base

In [ ]:
if BASE_NAME == 'BASE 1':
  PATH_BASE = "/content/drive/MyDrive/Base de dados/Base1JuntaHateENotHateDaLeila.csv"

if BASE_NAME == 'BASE 2':
  PATH_BASE = "/content/drive/MyDrive/Base de dados/Base2_maior4_menor25_limpissima.csv"

In [ ]:
tweets = pd.read_csv(PATH_BASE, index_col=0)
tweets

,text,label
1,retwet eu queria compartilhar mas antes eu gos...,not_hate
2,gostei da pulseira e da educação dele nossaaa ...,not_hate
3,esses não irão ao encontro e beijing,not_hate
4,cadê eles está faltando,not_hate
5,e eu me questionando quem é esse blogueiro sem...,not_hate
...,...,...
19277,estou sem presidente por quatro anos nãomerepr...,not_hate
19278,somos todos iguais braços dados ou não quem sa...,not_hate
19279,que deus nos proteja e cuidem um do outro somo...,not_hate
19280,quem será que não sabe votar,not_hate


In [ ]:
# Pega a quantidade de palavras únicas dos tweets de toda a base
_, text_tokenizer = tokenize(tweets['text'])
text_vocab_size = len(text_tokenizer.word_index)
print("Vocabulary size:", text_vocab_size)

Vocabulary size: 60417


# Prepara os dados de train, test e validation da base

In [ ]:
NUM_BASES = 2
# Carrega os paths das divisões das bases
LIST_PATH_TRAIN = []
LIST_PATH_TEST = []
LIST_PATH_VAL = []
for i in range(NUM_BASES):
  LIST_PATH_TRAIN.append('/content/drive/MyDrive/Base de dados/TCC/dados/processadas/bases_divididas/train'+str(i)+'.csv')
  LIST_PATH_TEST.append('/content/drive/MyDrive/Base de dados/TCC/dados/processadas/bases_divididas/test'+str(i)+'.csv')
  LIST_PATH_VAL.append('/content/drive/MyDrive/Base de dados/TCC/dados/processadas/bases_divididas/val'+str(i)+'.csv')

In [ ]:
# 0 - Base0 bruta não processada
# 1 - Base1 Com StopWords
# 2 - Base2 Sem StopWords
# 3 - Base2 Nossa sem stopwords
base_index = int(BASE_NAME.split()[1])
base_train = pd.read_csv(LIST_PATH_TRAIN[base_index], index_col=0)
base_test = pd.read_csv(LIST_PATH_TEST[base_index], index_col=0)
base_val = pd.read_csv(LIST_PATH_VAL[base_index], index_col=0)

In [ ]:
base_train.label = [1 if each == "hate" else 0 for each in base_train.label]
base_test.label = [1 if each == "hate" else 0 for each in base_test.label]
base_val.label = [1 if each == "hate" else 0 for each in base_val.label]

In [ ]:
# Dados train
Y_TRAIN = base_train['label']
base_train.drop(['label'], axis='columns', inplace=True)
X_TRAIN = base_train.to_numpy()

# Dados test
Y_TEST = base_test['label']
base_test.drop(['label'], axis='columns', inplace=True)
X_TEST = base_test.to_numpy()

# Dados validation
Y_VAL = base_val['label']
base_val.drop(['label'], axis='columns', inplace=True)
X_VAL = base_val.to_numpy()

In [ ]:
max_text_length = X_TRAIN.shape[1]

In [ ]:
# parametros estaticos:
EPOCHS = 10
EMBEDDING_DIMENSION = 300
MAX_TEXT_SIZE = max_text_length
VOCAB_SIZE = text_vocab_size

In [ ]:
def objective(trial):

  epochs_size = trial.suggest_int('epochs', 10, 32)
  batch_size = trial.suggest_int('batch_size', 32, 64)
  dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
  learning_rate = trial.suggest_float('learn_rate', 0.0001, 0.001)
  optimizer_name = trial.suggest_categorical('optmizer', ['nadam', 'rmsprop', 'sgd'])

  model = keras.Sequential([
  keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIMENSION, input_length=MAX_TEXT_SIZE),
  keras.layers.LSTM(MAX_TEXT_SIZE, dropout=dropout_rate, name="LSTM"),
  keras.layers.Dense(1, activation='sigmoid')
  ])

  if optimizer_name == '  nadam':
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate, name='nadam')
  elif optimizer_name == 'rmsprop':
    opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate, name='rmsprop')
  else:
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, name='sgd')

  model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=opt, metrics=['accuracy'])

  ra_val = RocAucEvaluation(validation_data=(X_VAL, Y_VAL), interval = 1)

  csv_logger = CSVLogger('log.csv', append=False, separator=';')

  early_stop = EarlyStopping(monitor = "val_loss", mode = "min", verbose = 0, patience = 1)

  history = model.fit(X_TRAIN, Y_TRAIN, batch_size=batch_size, epochs=epochs_size, validation_data=(X_VAL, Y_VAL), verbose=0, use_multiprocessing=True)
  score = model.evaluate(X_TEST, Y_TEST, verbose=0)

  return score[0]

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2023-06-15 17:39:17,891] A new study created in memory with name: no-name-c9d7dc9a-10c6-4b76-8662-b041becb1d6f
[I 2023-06-15 17:40:01,820] Trial 0 finished with value: 0.4007960259914398 and parameters: {'epochs': 13, 'batch_size': 58, 'dropout_rate': 0.1633881224733011, 'learn_rate': 0.0003670091419076057, 'optmizer': 'nadam'}. Best is trial 0 with value: 0.4007960259914398.
[I 2023-06-15 17:40:53,235] Trial 1 finished with value: 0.18303783237934113 and parameters: {'epochs': 15, 'batch_size': 51, 'dropout_rate': 0.22368609687824098, 'learn_rate': 0.00040392055437058546, 'optmizer': 'rmsprop'}. Best is trial 0 with value: 0.4007960259914398.
[I 2023-06-15 17:42:01,962] Trial 2 finished with value: 0.3551628887653351 and parameters: {'epochs': 28, 'batch_size': 52, 'dropout_rate': 0.252670744791964, 'learn_rate': 0.0003731770250523753, 'optmizer': 'sgd'}. Best is trial 0 with value: 0.4007960259914398.
[I 2023-06-15 17:42:54,773] Trial 3 finished with value: 0.35348305106163025 and

# Aplicando o SMOTE

In [ ]:
print('Dataset shape without resampling %s' % Counter(Y_TRAIN))
print('Dataset shape without resampling %s' % Counter(Y_VAL))
print('Dataset shape without resampling %s' % Counter(Y_TEST))

Dataset shape without resampling Counter({0: 11570, 1: 1465})
Dataset shape without resampling Counter({0: 3856, 1: 489})
Dataset shape without resampling Counter({0: 3856, 1: 489})


In [ ]:
smote = SMOTE(random_state=0)
X_TRAIN, Y_TRAIN = smote.fit_resample(X_TRAIN, Y_TRAIN)
X_VAL, Y_VAL = smote.fit_resample(X_VAL, Y_VAL)
X_TEST, Y_TEST = smote.fit_resample(X_VAL, Y_VAL)

In [ ]:
print('Dataset shape without resampling %s' % Counter(Y_TRAIN))
print('Dataset shape without resampling %s' % Counter(Y_VAL))
print('Dataset shape without resampling %s' % Counter(Y_TEST))

Dataset shape without resampling Counter({0: 11570, 1: 11570})
Dataset shape without resampling Counter({0: 3856, 1: 3856})
Dataset shape without resampling Counter({0: 3856, 1: 3856})


In [ ]:
max_text_length = X_TRAIN.shape[1]

In [ ]:
def objective(trial):

  # Define the hyperparameters to optimize
  epochs_size = trial.suggest_int('epochs', 10, 32)
  batch_size = trial.suggest_int('batch_size', 32, 64)
  dropout_rate = trial.suggest_float('dropout_rate', 0.0, 0.5)
  learning_rate = trial.suggest_float('learn_rate', 0.0001, 0.001)
  optimizer_name = trial.suggest_categorical('optmizer', ['nadam', 'rmsprop', 'sgd'])

  # Define the model architecture
  model = keras.Sequential([
  keras.layers.Embedding(VOCAB_SIZE, EMBEDDING_DIMENSION, input_length=MAX_TEXT_SIZE),
  keras.layers.LSTM(MAX_TEXT_SIZE, dropout=dropout_rate, name="LSTM"),
  keras.layers.Dense(1, activation='sigmoid')
  ])

  if optimizer_name == '  nadam':
    opt = tf.keras.optimizers.Adam(learning_rate=learning_rate, name='nadam')
  elif optimizer_name == 'rmsprop':
    opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate, name='rmsprop')
  else:
    opt = tf.keras.optimizers.SGD(learning_rate=learning_rate, name='sgd')

  # Compile the model with the hyperparameters
  model.compile(loss=tf.keras.losses.BinaryCrossentropy(), optimizer=opt, metrics=['accuracy'])

  # Train the model and compute the validation accuracy
  ra_val = RocAucEvaluation(validation_data=(X_VAL, Y_VAL), interval = 1)

  csv_logger = CSVLogger('log.csv', append=False, separator=';')

  early_stop = EarlyStopping(monitor = "val_loss", mode = "min", verbose = 0, patience = 1)

  history = model.fit(X_TRAIN, Y_TRAIN, batch_size=batch_size, epochs=epochs_size, validation_data=(X_VAL, Y_VAL), verbose=0, use_multiprocessing=True,  callbacks = [ra_val, csv_logger, early_stop])
  score = model.evaluate(X_TEST, Y_TEST, verbose=0)

  # Return the score to optimize
  return score[0]

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=20)

[I 2023-06-15 18:30:20,149] A new study created in memory with name: no-name-375625a1-6bdc-4cd5-b5d5-d876257ef132
[I 2023-06-15 18:33:46,263] Trial 0 finished with value: 0.6884074211120605 and parameters: {'epochs': 31, 'batch_size': 35, 'dropout_rate': 0.23864417359588114, 'learn_rate': 0.0008237391431999315, 'optmizer': 'nadam'}. Best is trial 0 with value: 0.6884074211120605.
[I 2023-06-15 18:34:14,819] Trial 1 finished with value: 0.39844605326652527 and parameters: {'epochs': 20, 'batch_size': 49, 'dropout_rate': 0.10999051375095165, 'learn_rate': 0.0005866287099073257, 'optmizer': 'rmsprop'}. Best is trial 0 with value: 0.6884074211120605.
[I 2023-06-15 18:35:52,234] Trial 2 finished with value: 0.6924305558204651 and parameters: {'epochs': 18, 'batch_size': 43, 'dropout_rate': 0.43743064470714976, 'learn_rate': 0.0009737974821009083, 'optmizer': 'nadam'}. Best is trial 2 with value: 0.6924305558204651.
[I 2023-06-15 18:38:16,681] Trial 3 finished with value: 0.6932447552680969 